In [ ]:
import functools
import pathlib

import lifetimes

In [ ]:
local_path = pathlib.Path("/home/fabian/Documents/MAELSTROM/data/pca/")
outfile = f"{local_path}/temperature_level_128_daily_averages_2017_2020.nc"
paths = [p.as_posix() for p in sorted(list(local_path.glob("ml*.nc")))]
paths

In [ ]:
preprocess = functools.partial(
    lifetimes.features.methods.select_model_level_and_calculate_daily_mean,
    level=128,
)
ds = lifetimes.features.EcmwfIfsHresDataset(
    paths=paths,
    overlapping=True,
    preprocessing=preprocess,
)

In [ ]:
ds = ds.as_xarray(drop_variables=["q", "u", "v"])
ds["time"]

In [ ]:
ds.to_netcdf(outfile)